In [1]:
#!/usr/bin/env python2

from __future__ import division
from __future__ import with_statement
from __future__ import print_function

import numpy as np
import deepmodels
import json
import time
import argparse
import os.path
import subprocess
import imageutils
import utils

# use deepmodels_torch_facenet.py for facenet features instead of vgg features
import deepmodels_torch


In [2]:
# code to change image such that the source image looks more like the target image

In [3]:
with open('datasets/lfw/lfw_binary_attributes.json') as f: lfw=json.load(f)
with open('datasets/lfw/filelist.txt','r') as f: lfw_filelist=['images/'+x.strip() for x in f.readlines()]

In [4]:
backend = 'torch'
device_id = 0
K = 100
scaling = 'beta'
itera = 500
postprocess=set('color'.split(','))
lfw_path = "/home/vk352/FaceDetection/datasets/lfw"

In [5]:
# images of target person 
lfw_target = ["images/lfw/George_W_Bush/"+f for f in os.listdir(lfw_path+'/George_W_Bush') if os.path.isfile(os.path.join(lfw_path+'/George_W_Bush', f))]

In [7]:
minimum_resolution=200
model=deepmodels_torch.vgg19g_torch(device_id=device_id)

In [8]:
# preprocessed lfw images that have the aligned face without the background
if not os.path.exists('images/lfw_aegan'):
    url='https://www.dropbox.com/s/isz4ske2kheuwgr/lfw_aegan.tar.gz?dl=1'
    subprocess.check_call(['wget',url,'-O','lfw_aegan.tar.gz'])
    subprocess.check_call(['tar','xzf','lfw_aegan.tar.gz'])
    subprocess.check_call(['rm','lfw_aegan.tar.gz'])

In [9]:
# list of sounce image that needs to be changed. All souce images will be made like the average of the target image
X = ['images/lfw/Tom_Hanks/Tom_Hanks_0005.jpg']

In [10]:
K=200
delta = '0.4'
delta_params=[float(x.strip()) for x in delta.split(',')]
# t0=time.time()
result=[]
original=[]

In [11]:
for i in range(len(X)):
    result.append([])
    xX=X[i].replace('lfw','lfw_aegan')
    o=imageutils.read(xX)

In [12]:
image_dims=o.shape[:2]

In [13]:
if min(image_dims)<minimum_resolution:
    s=float(minimum_resolution)/min(image_dims)
    image_dims=(int(round(image_dims[0]*s)),int(round(image_dims[1]*s)))
    o=imageutils.resize(o,image_dims)

In [14]:
XF=model.mean_F([o])

/home/vk352/FaceDetection/deepfeatinterp/deepmodels_torch.py:137: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input_var = Variable(input, volatile=True).cuda()


In [15]:
original.append(o)

In [16]:
# the positive images are the images in the target
xP = [x.replace('lfw','lfw_aegan') for x in lfw_target]
# the negative images are random images. We can also try changing this to images of the source person
xQ=[lfw_filelist[i].replace('lfw','lfw_aegan') for i in np.random.choice(len(lfw_filelist), K, replace=False)]

In [17]:
PF=model.mean_F(utils.image_feed(xP[:K],image_dims))
QF=model.mean_F(utils.image_feed(xQ[:K],image_dims))

In [18]:
if scaling=='beta':
    WF=(PF-QF)/((PF-QF)**2).mean()
elif scaling=='none':
    WF=(PF-QF)
max_iter=itera
init=o

In [19]:
# reconstruct vector XF+delta*WF to image for different deltas
for delta in [0.1, 0.2, 0.3, 0.4]:
    print(xX,delta)
    t2=time.time()
    Y=model.F_inverse(XF+delta*WF,max_iter=max_iter,initial_image=init)
#     Y=model.F_inverse(PF,max_iter=max_iter,initial_image=init)
    t3=time.time()
    print('{} minutes to reconstruct'.format((t3-t2)/60.0))
    result[-1].append(Y)
    max_iter=itera//2
    init=Y

images/lfw_aegan/Tom_Hanks/Tom_Hanks_0005.jpg 0.1


/home/vk352/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


0.24623467922210693 minutes to reconstruct
images/lfw_aegan/Tom_Hanks/Tom_Hanks_0005.jpg 0.2
0.132330850760142 minutes to reconstruct
images/lfw_aegan/Tom_Hanks/Tom_Hanks_0005.jpg 0.3
0.12034370104471842 minutes to reconstruct
images/lfw_aegan/Tom_Hanks/Tom_Hanks_0005.jpg 0.4
0.12156476179758707 minutes to reconstruct


In [20]:
result=np.asarray(result)
original=np.asarray(original)

In [21]:
if 'color' in postprocess:
    result=utils.color_match(np.expand_dims(original,1),result)

Computing color match (1, 1, 200, 200, 3) (1, 4, 200, 200, 3)


In [22]:
m=imageutils.montage(np.concatenate([np.expand_dims(original,1),result],axis=1))

In [23]:
imageutils.write('results/coll0.4.png',m)